In [2]:
import numpy as np
import pandas as pd
from typing import Tuple, List, Dict

### Multi-site and multi-component extension of the McLean isotherm

 $$c_{i,k}^{GB} = \frac{c_i \times \exp(-\beta \times E_{i,k}^{seg})}{1 + \sum_{j} c_j \times (\exp(-\beta \times E_{j,k}^{seg} - 1))}$$
 The indices $i$ and $j$ run over all components and the index $k$ denotes the sites at the grain boundary.
 $c_i$ denotes the bulk content of component $i$. The total GB content of solute $i$ can be calulated with $c_{i}^{GB} = \frac{1}{N}\sum_kc_{i,k}^{GB}$. Hence, the full equation is given by
 $$c_{i}^{GB} = \frac{1}{N}\sum_k \frac{c_i \times \exp(-\beta \times E_{i,k}^{seg})}{1 + \sum_{j} c_j \times (\exp(-\beta \times E_{j,k}^{seg} - 1))}$$
 where N is the total number of sites $k$. $\beta$ is given by $\frac{1}{k_B*T}$.

 The interfacial excess (IFE) can be calculated through
 $$IFE=N(c_i^{GB}-c{_i})/A$$

In [3]:
# get segregation energies for a given element
def seg_energies(element: str, data_frame: pd.DataFrame) -> List:
    segs = []
    for i in data_frame.loc[:, element]:
        segs.append(i)
    return segs

# function which returns 1/(k_B*T)
def beta(T: float) -> float:
    # gas constant in eV/K --> Boltzmann constant
    k_B=(8.314/(1.602*10**(-19)))/(6.022*10**(23)) 
    return 1/(k_B*T)


In [3]:
def mclean(A: float, x_b: List[float], temp: Tuple[float, float, float] | float, 
           sites_multiplicity: List[int], segregation_energies: np.ndarray) -> Tuple[Dict[str, List], Dict[str, List], List]: 
    
    """Solves the multi-site and multi-component extension of the McLean equation.

    Parameters
    ----------
    A : float
        The GB area in nm.
    x_b : List[float]
        Concentrations of components. 
    temp : Tuple[float, float, float] | float
        Temperature range or specific temperature.
    sites_multiplicity : List[int]
        The multiplicity of a GB site shows how often one and the same site occurs in the GB. 
    segregation_energies : np.ndarray
        The segregation energies of each component to each site in the GB.

    Returns
    ------- 
    Tuple[Dict[str, List], Dict[str, List], List]
        GB concentrations and IFE values of each component.
        Additionally, it returns the temperature profile.   
    """
    

    total_sites = np.sum(sites_multiplicity) # total number of sites
    if isinstance(temp, (float, int)):
        Tstart = temp
        Tend = Tstart + 1.0
        dT = 1.0
    else:
        Tstart, Tend, dT = temp

    # variables for storing results
    c_gb = dict()
    IFE = dict()
    temps = list()

    
    for Temp in np.arange(Tstart, Tend, dT):
        temps.append(Temp)
        beta_ = beta(T=Temp)
        for comp in range(len(x_b)):
            frac = 0 

            for i in range(len(sites_multiplicity)):
                m = sites_multiplicity[i]
                sum_in_denom = 0
                nom = x_b[comp] * np.exp(-beta_ * segregation_energies[comp, i]) # nominator is fixed for a specific component and a single site

                for k in range(len(x_b)): # summing over all components, because in the nominator we have the sum over all c_k*(exp(-beta*E_i_k) - 1)
                    denom_sum_part = x_b[k] * ( np.exp(-beta_ * segregation_energies[k,i]) - 1 )
                    sum_in_denom += denom_sum_part
                
                frac += m*(nom/(1+sum_in_denom))

            gb_comp = frac/total_sites
            ife = total_sites*(gb_comp-x_b[comp])/A

            key = f'{comp}'
            if key not in c_gb or key not in IFE:
                c_gb[key] = list()
                IFE[key] = list()
            
            c_gb[key].append(gb_comp)
            IFE[key].append(ife)

    return c_gb, IFE, temps